## ¿Qué es el Proceso de WebScrapping?

![imagen de webscrapping](assets/webscrapping.jpg)


Es el proceso para extraer datos desde un sitio web.

Muchas veces distintos sitios web comparten datos sobre algo que nos interesa o sirve, sin embargo extraerlo manualmente puede ser tedioso y hasta complicado. 

Luego, una opción viable que tenemos es extraer dicha información usando librerías de Python: **Requests y BeautifulSoup**.





In [13]:
import requests as r
from bs4 import BeautifulSoup as bs
import json
import pandas as pd


Usaremos como ejemplo la extracción de datos de un tribunal ambiental.

Contexto: Chile posee legislación frente a asuntos relacionados al medio ambiente o de comunidades indígenas, y estos son resueltos por los tribunales ambientales, quienes poseen expertos en el área. Cada uno de los casos que el tribunal ambiental litiga, son guardados en los expedientes del tribunal como 'causas'.


# Paso 1: Analizar de donde viene la información

Normalmente, existen dos tipos de sitios web: estáticos y dinámicos. Los estáticos se basan en que tienen la información contenida en la misma página web, mientras que las dinámicas tienen que hacer llamadas a un servidor web mediante un protocolo HTTP.

Puesto que los datos rara vez son estáticos, y quienes los administrar los suelen modificar muchas veces, es muy común que dichas páginas web sean **dinamicas**.

## Ahora cabe la pregunta: ¿Cómo se comunica el sitio web con el servidor y viceversa?

con **POST y GET** : son dos de los métodos más comunes del protocolo HTTP que los navegadores y servidores web utilizan para comunicarse.


- GET: se utiliza principalmente para solicitar datos de un servidor.
- POST: para enviar datos al servidor para ser procesados.

**Para este ejemplo usaremos el tribunal 1A: https://www.portaljudicial1ta.cl/sgc-web/consulta-causa.html**

![imagen de webscrapping](assets/tribunal1A.png)

Para observar estos métodos, necesitaremos un explorador web con herramientas de desarrollador (Developer tools). **Google Chrome** es un buen ejemplo de esto. Para acceder a ella hacemos click derecho y damos a **inspeccionar**

Puedes filtrar los datos con las herramientas propias de la página web a tu gusto, luego esto generará una llamada al servidor. Cuando carguen los datos, en la ventana de dev tools debería aparecer algo así (A veces pueden ser multiples llamadas, este ejemplo es simple)

Las tres cosas que nos interesan son: El headers, el payload y el response.

## El headers nos da información sobre el método empleado para cargar los datos
![imagen de webscrapping](assets/post.png)

## el Payload son las instrucciones especificas que entrega la página web al servidor para recibir datos (Aquí por ejemplo se puedes especificar filtros)

![imagen de webscrapping](assets/payload.png)
## El response es la respuesta, es decir, los datos obtenidos.

![imagen de webscrapping](assets/response.png)








## Paso 2: Una vez chequeamos que efectivamente esta llamada al servidor tiene los datos que nos interesa, es momento de codear!

## Definimos la url.

In [3]:
url = 'https://www.portaljudicial1ta.cl/sgc-ws/rest/consulta-causa/get-consulta-causa'

# Definimos el Header

In [4]:
headers = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Accept-Language': 'en-US,en;q=0.9',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Host': 'www.portaljudicial1ta.cl',
    'Origin': 'https://www.portaljudicial1ta.cl',
    'Referer': 'https://www.portaljudicial1ta.cl/sgc-web/consulta-causa.html',
    'Sec-Ch-Ua': '"Chromium";v="128", "Not;A=Brand";v="24", "Google Chrome";v="128"',
    'Sec-Ch-Ua-Mobile': '?0',
    'Sec-Ch-Ua-Platform': '"Windows"',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest'
}


# Definimos el Payload

In [5]:
payload = {'tipoCausa': 'null', #Definimos los parametros que se enviaran en la peticion (en este caso, los parametros son nulos lo cual indica que queremos todos los datos)
'numeroRol':'',
'anioIngreso': 'null',
'estado': 'null'
}

# Simulamos la llamada al servidor con la libreria Requests

In [7]:
response = r.post(url, headers=headers, data=payload)
response

<Response [200]>

# Ahora usamos la librería jSON para procesar el response

In [8]:
jsonformat = response.json()
jsonformat 

{'message': 'OK (2024-08-23T21:21:26.661037)',
 'response': '[{"caratula":"Superintendencia del Medio Ambiente con Taruca Solar SpA","estado":"Terminadas ( Resuelta) ","fechaCausa":"19-07-2024 04:00","idCausa":"4d77775e-e138-4c2d-a98b-8f6265687673","idEstado":0,"idPendiente":0,"numeroRol":"S-24-2024","realizada":false,"subTipoCausa":"Resuelta","tipoCausa":"Solicitud"},{"caratula":"Soledad de la Cruz Fuentealba Triviño y otros con Servicio de Evaluación Ambiental ","estado":"En tramitación ( Espera de Informe) ","fechaCausa":"15-07-2024 04:00","idCausa":"4e368a1d-f7be-49a6-b63b-12093a787736","idEstado":0,"idPendiente":0,"numeroRol":"R-108-2024","realizada":false,"subTipoCausa":"Espera de Informe","tipoCausa":"Reclamación"},{"caratula":"Consejo de Defensa del Estado  con Quiborax S.A.","estado":"En tramitación ( Demanda) ","fechaCausa":"02-07-2024 04:00","idCausa":"ddfa0e93-8c99-4b06-809f-aa078169fb56","idEstado":0,"idPendiente":0,"numeroRol":"D-31-2024","realizada":false,"subTipoCausa":

Como podrás observar, el response es un diccionario con dos llaves: message (Que corresponde a la respuesta del servidor frente a nuestra solicitud) y response, que corresponde a los datos que solicitamos.

Nos quedamos con este último.

Ok, pero si ingresamos la llave, tenemos el problema de que el resultado es una string de una lista que contiene muchos subdiccionarios, lo cual es engorroso de transformar.

In [10]:
jsonformat['response']

'[{"caratula":"Superintendencia del Medio Ambiente con Taruca Solar SpA","estado":"Terminadas ( Resuelta) ","fechaCausa":"19-07-2024 04:00","idCausa":"4d77775e-e138-4c2d-a98b-8f6265687673","idEstado":0,"idPendiente":0,"numeroRol":"S-24-2024","realizada":false,"subTipoCausa":"Resuelta","tipoCausa":"Solicitud"},{"caratula":"Soledad de la Cruz Fuentealba Triviño y otros con Servicio de Evaluación Ambiental ","estado":"En tramitación ( Espera de Informe) ","fechaCausa":"15-07-2024 04:00","idCausa":"4e368a1d-f7be-49a6-b63b-12093a787736","idEstado":0,"idPendiente":0,"numeroRol":"R-108-2024","realizada":false,"subTipoCausa":"Espera de Informe","tipoCausa":"Reclamación"},{"caratula":"Consejo de Defensa del Estado  con Quiborax S.A.","estado":"En tramitación ( Demanda) ","fechaCausa":"02-07-2024 04:00","idCausa":"ddfa0e93-8c99-4b06-809f-aa078169fb56","idEstado":0,"idPendiente":0,"numeroRol":"D-31-2024","realizada":false,"subTipoCausa":"Demanda","tipoCausa":"Demanda"},{"caratula":"Comunidad Indí

Afortunadamente, la libreria jSON tiene todo esto abarcado. Usaremos el metodo Loads. Con esto convertimos esa string en una lista que soporta las operaciones y metodos de python.

In [12]:
to_list=json.loads(jsonformat['response'])
to_list

[{'caratula': 'Superintendencia del Medio Ambiente con Taruca Solar SpA',
  'estado': 'Terminadas ( Resuelta) ',
  'fechaCausa': '19-07-2024 04:00',
  'idCausa': '4d77775e-e138-4c2d-a98b-8f6265687673',
  'idEstado': 0,
  'idPendiente': 0,
  'numeroRol': 'S-24-2024',
  'realizada': False,
  'subTipoCausa': 'Resuelta',
  'tipoCausa': 'Solicitud'},
 {'caratula': 'Soledad de la Cruz Fuentealba Triviño y otros con Servicio de Evaluación Ambiental ',
  'estado': 'En tramitación ( Espera de Informe) ',
  'fechaCausa': '15-07-2024 04:00',
  'idCausa': '4e368a1d-f7be-49a6-b63b-12093a787736',
  'idEstado': 0,
  'idPendiente': 0,
  'numeroRol': 'R-108-2024',
  'realizada': False,
  'subTipoCausa': 'Espera de Informe',
  'tipoCausa': 'Reclamación'},
 {'caratula': 'Consejo de Defensa del Estado  con Quiborax S.A.',
  'estado': 'En tramitación ( Demanda) ',
  'fechaCausa': '02-07-2024 04:00',
  'idCausa': 'ddfa0e93-8c99-4b06-809f-aa078169fb56',
  'idEstado': 0,
  'idPendiente': 0,
  'numeroRol': 'D-

## Ahora esto lo podemos transformar facilmente a un dataset de pandas y guardarlo en el formato que queramos.

In [14]:
df = pd.DataFrame(to_list)
df

,caratula,estado,fechaCausa,idCausa,idEstado,idPendiente,numeroRol,realizada,subTipoCausa,tipoCausa
0,Superintendencia del Medio Ambiente con Taruca...,Terminadas ( Resuelta),19-07-2024 04:00,4d77775e-e138-4c2d-a98b-8f6265687673,0,0,S-24-2024,False,Resuelta,Solicitud
1,Soledad de la Cruz Fuentealba Triviño y otros ...,En tramitación ( Espera de Informe),15-07-2024 04:00,4e368a1d-f7be-49a6-b63b-12093a787736,0,0,R-108-2024,False,Espera de Informe,Reclamación
2,Consejo de Defensa del Estado con Quiborax S.A.,En tramitación ( Demanda),02-07-2024 04:00,ddfa0e93-8c99-4b06-809f-aa078169fb56,0,0,D-31-2024,False,Demanda,Demanda
3,Comunidad Indígena Atacameña Conchí Viejo con...,En tramitación ( Demanda),03-06-2024 04:00,5e774c61-60f6-49ec-8939-41133ad8067e,0,0,D-30-2024,False,Demanda,Demanda
4,Superintendencia del Medio Ambiente con Minera...,Terminadas ( Resuelta),31-05-2024 04:00,32c8b5a5-1b47-482b-ab13-cafb6e363592,0,0,S-23-2024,False,Resuelta,Solicitud
...,...,...,...,...,...,...,...,...,...,...
165,Compañía Minera del Pacífico S.A. con Superint...,En tramitación ( Demanda),14-12-2017 03:00,fb45290b-f58c-4b39-8ac6-a10781ab30f4,0,0,R-2-2017,False,Demanda,Reclamación
166,Superintendencia del Medio Ambiente con SQM S.A.,En tramitación ( Demanda),04-12-2017 03:00,73f5aba4-5e74-4243-a94e-0499329511eb,0,0,S-2-2017,False,Demanda,Solicitud
167,Andes Iron SpA con Servicio de Evaluación Ambi...,En tramitación ( Demanda),01-12-2017 03:00,715bb837-ac88-418b-bf20-e1b95c7a85ac,0,0,R-1-2017,False,Demanda,Reclamación
168,Superintendencia del Medio Ambiente con Ibereo...,En tramitación ( Demanda),30-11-2017 03:00,2de27319-1447-4256-8b17-13594c2a9e54,0,0,S-1-2017,False,Demanda,Solicitud


# Bien, pero ... ¿Qué pasa si quiero información especifica de una página?

Bueno, en ese caso, tendremos que explorar el html de la página web en cuestión. Imagina que queremos extraer el contenido de una noticia que nos interesó del sitio web 'El Mostrador'. 

Ingresamos la url y hacemos un response directamente en la url usando el método get. Esto nos permitirá descargar el html completo.

**Warning: Este metodo no siempre puede funcionar, ya que depende de como esté desarrollado el sitio web. Algunos utilizan códigos de Javascript para renderizarse y la librería requests no soporta el renderizado de páginas web. En dichos casos es recomendable usar Selenium**

In [37]:
url = 'https://www.elmostrador.cl/cultura/2024/08/23/voz-de-trompeta-el-cortometraje-chileno-sin-dialogo-que-logra-exito-en-los-festivales-del-mundo/'

response = r.get(url, headers=headers)

## Ahora, usamos la libreria BeautifulSoup para poder parsear el contenido html obtenido.

In [38]:
soup = bs(response.content, 'html.parser')
soup


<!DOCTYPE html>

<html class="u-disable-effects scheme-desktop" lang="es">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="" name="description"/>
<meta content="" name="author"/>
<link href="/favicon.png" rel="shortcut icon" type="image/x-icon"/>
<link href="/_templates/desktop/includes/css/main.min.css?v=1.0.0.52" rel="stylesheet"/>
<meta content="" name="theme-color"/>
<!-- VARIABLES PARA METAS -->
<!-- /VARIABLES PARA METAS -->
<title>“Voz de Trompeta”, el cortometraje chileno sin diálogo que logra éxito en los festivales del mundo</title>
<meta content="“Voz de Trompeta”" name="keywords"/>
<meta content="Realizará un estreno el 27 de septiembre en el Club de Jazz Estrella Negra de Valparaíso, que contará con la exhibición del cortometraje y presentación en vivo de la banda sonora." name="description"/>
<meta content="website" property="og:type">
<meta content=

## Buscamos los datos que nos interesan en el html (Usamos el dev tools en Elements)

![imagen de webscrapping](assets/mostrador.png)

Este proceso es tedioso, pero efectivo. Simplemente debemos ir buscando hasta encontrar el elemento de html (En este caso es un párrafo con una clase con un determinado nombre) y luego usando BeautifulSoup lo filtramos.

In [40]:
data = soup.find_all('p', class_='d-the-single__excerpt | u-fw-600')
data

[<p class="d-the-single__excerpt | u-fw-600">
                           Realizará un estreno el 27 de septiembre en el Club de Jazz Estrella Negra de Valparaíso, que contará con la exhibición del cortometraje y presentación en vivo de la banda sonora.
                       </p>]

Podemos limpiar aun más el dato.

In [43]:
data[0].text

'\n                          Realizará un estreno el 27 de septiembre en el Club de Jazz Estrella Negra de Valparaíso, que contará con la exhibición del cortometraje y presentación en vivo de la banda sonora.\n                      '

In [45]:
limpio = data[0].text.replace('\n', '')
limpio = " ".join(limpio.split())
limpio

'Realizará un estreno el 27 de septiembre en el Club de Jazz Estrella Negra de Valparaíso, que contará con la exhibición del cortometraje y presentación en vivo de la banda sonora.'